In [37]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import StratifiedKFold
from sklearn.naive_bayes import GaussianNB
from sklearn.cross_validation import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import GridSearchCV
import xgboost as xgb
import numpy as np

In [4]:
data_train = pd.read_table('../data/products_sentiment_train.tsv', header=None, index_col=False)

In [5]:
data_train['text'] = data_train[0]
data_train['label'] = data_train[1]
del data_train[0]
del data_train[1]

In [6]:
data_train.head()

,text,label
0,"2 . take around 10,000 640x480 pictures .",1
1,i downloaded a trial version of computer assoc...,1
2,the wrt54g plus the hga7t is a perfect solutio...,1
3,i dont especially like how music files are uns...,0
4,i was using the cheapie pail ... and it worked...,1


In [13]:
X = data_train['text']
y = data_train['label']

In [8]:
count_vectorizer = CountVectorizer(ngram_range=(1, 3))
X_vect = count_vectorizer.fit_transform(X).toarray()
estimator = LogisticRegression()

scores = cross_val_score(estimator=estimator, X=X_vect, y=y, cv=20)
print round(scores.mean(), 4)

0.771


In [9]:
# cv = StratifiedKFold(y, n_folds=3, shuffle=True, random_state=1)

# ald_xgb_model = xgb.XGBClassifier()
# ald_xgb_params = [
#         {
#          "n_estimators": [700],
#          "max_depth": [12],
#          "learning_rate": [0.005]
#         }
# ]
# alg_frst_grid = GridSearchCV(ald_xgb_model, ald_xgb_params, cv=cv, refit=True, verbose=1, n_jobs=4)
# alg_frst_grid.fit(X_vect, y)
# alg_frst_best = alg_frst_grid.best_estimator_
# print("Accuracy (random forest auto): {} with params {}"
#       .format(alg_frst_grid.best_score_, alg_frst_grid.best_params_))

In [29]:
cv = StratifiedKFold(y, n_folds=3, shuffle=True, random_state=1)

pipeline = Pipeline([
    ("vectorizer", CountVectorizer()), ("classifier", LogisticRegression())
])
pipeline_params = [{
    "vectorizer__stop_words": ['english', None]
}]

grid = GridSearchCV(pipeline, pipeline_params, cv=cv, refit=True, verbose=1, n_jobs=4)
grid.fit(X, y)
best = grid.best_estimator_
print("Accuracy (CountVectorizer + LogisticRegression): {} with params {}"
      .format(grid.best_score_, grid.best_params_))

Fitting 3 folds for each of 2 candidates, totalling 6 fits
Accuracy (CountVectorizer + LogisticRegression): 0.7545 with params {'vectorizer__stop_words': None}


[Parallel(n_jobs=4)]: Done   6 out of   6 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   6 out of   6 | elapsed:    0.2s finished


In [38]:
cv = StratifiedKFold(y, n_folds=200, shuffle=True, random_state=1)

pipeline = Pipeline([
    ("vectorizer", CountVectorizer()), ("classifier", GaussianNB())
])
pipeline_params = [{
    "vectorizer__stop_words": ['english', None]
}]

grid = GridSearchCV(pipeline, pipeline_params, cv=cv, refit=True, verbose=1, n_jobs=4)
grid.fit(X, y)
best = grid.best_estimator_
print("Accuracy (CountVectorizer + SGDClassifier): {} with params {}"
      .format(grid.best_score_, grid.best_params_))

Fitting 200 folds for each of 2 candidates, totalling 400 fits


JoblibTypeError: JoblibTypeError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/opt/local/anaconda/lib/python2.7/runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    169     pkg_name = mod_name.rpartition('.')[0]
    170     main_globals = sys.modules["__main__"].__dict__
    171     if alter_argv:
    172         sys.argv[0] = fname
    173     return _run_code(code, main_globals, None,
--> 174                      "__main__", fname, loader, pkg_name)
        fname = '/opt/local/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py'
        loader = <pkgutil.ImpLoader instance>
        pkg_name = ''
    175 
    176 def run_module(mod_name, init_globals=None,
    177                run_name=None, alter_sys=False):
    178     """Execute a module's code without importing it

................................................